# Examen 1 mod 3

## Importar

In [1]:
# %load basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
liga = '/home/ef/Documents/Diplomado/EXAMEN0301/'

In [2]:
import os
os.listdir(liga)

['datos-20210123T141747Z-001.zip', 'datos_restaurantes']

import shutil

shutil.unpack_archive(liga + os.listdir(liga)[0], 'datos_restaurantes', 'zip')

In [3]:
liga += 'datos_restaurantes/datos/'
os.listdir(liga)

['restaurants.csv',
 'parking.csv',
 'payment_methods.csv',
 'ratings.csv',
 'userpayment.csv',
 'users.csv',
 'hours.csv',
 'cuisine.csv',
 'usercuisine.csv']

In [4]:
resto = pd.read_csv(liga + 'restaurants.csv'
                   ).merge(pd.read_csv(liga + 'cuisine.csv'),
                           on = 'placeID', 
                           how = 'left'
                          ).merge(pd.read_csv(liga + 'hours.csv'), 
                                  on = 'placeID',
                                  how = 'left'
                                 ).drop_duplicates().set_index('placeID')
resto.columns

Index(['latitude', 'longitude', 'the_geom_meter', 'name', 'address', 'city',
       'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services', 'Rcuisine', 'hours', 'days'],
      dtype='object')

In [5]:
users = pd.read_csv(liga + 'users.csv').merge(pd.read_csv(liga + 'usercuisine.csv'), 
                                              on = 'userID', 
                                              how = 'left').drop_duplicates().set_index('userID')
users.columns

Index(['latitude', 'longitude', 'smoker', 'drink_level', 'dress_preference',
       'ambience', 'transport', 'marital_status', 'hijos', 'birth_year',
       'interest', 'personality', 'religion', 'activity', 'color', 'weight',
       'budget', 'height', 'Rcuisine'],
      dtype='object')

In [6]:
ratings = pd.read_csv(liga + 'ratings.csv').drop_duplicates()
ratings.sample(4)

,userID,placeID,rating,food_rating,service_rating
826,U1003,135041,0,1,1
342,U1094,135108,0,1,1
129,U1026,132665,2,2,2
102,U1066,135019,2,1,0


## Restaurante

In [7]:
rest = resto.copy()
resto.drop(columns = ['the_geom_meter', 
                      'name', 'address','state', 'country', 
                      'fax', 'zip','dress_code', 'url',
                      'Rambience','other_services'], inplace = True)
resto.sample(4)

,latitude,longitude,city,alcohol,smoking_area,accessibility,price,franchise,area,Rcuisine,hours,days
placeID,,,,,,,,,,,,
132866,22.141220,-100.931311,San Luis Potosi,No_Alcohol_Served,not permitted,completely,medium,f,closed,Cafeteria,09:00-22:00;,Sun;
135066,22.168350,-100.972466,San Luis Potosi,Wine-Beer,section,no_accessibility,high,f,closed,NaN,07:00-23:00;,Sun;
135040,22.135617,-100.969709,San Luis Potosi,Wine-Beer,none,no_accessibility,high,f,closed,NaN,08:00-23:30;,Sun;
135057,22.145992,-100.955118,San Luis Potosi,Wine-Beer,section,no_accessibility,medium,f,closed,Bar,13:00-23:00;,Sat;


In [8]:
import re
resto['city'] = resto['city'].str.replace('.','').str.strip().str.lower()
resto['city'] = [re.sub(r'^san\sluis\s\w+','slp',x) for x in 
                 [re.sub(r'[\w+\s]*victoria$','cd victoria',x) for x in resto['city']]]
resto['city'].value_counts(1)

slp            0.595843
?              0.124711
cuernavaca     0.120092
cd victoria    0.110855
jiutepec       0.027714
soledad        0.020785
Name: city, dtype: float64

In [9]:
resto['smoking_area'] = resto['smoking_area'].replace({'none':'not permitted',
                                                       'section':'permitted',
                                                       'only at bar':'permitted'})
resto['smoking_area'].value_counts(1)

not permitted    0.709007
permitted        0.290993
Name: smoking_area, dtype: float64

In [10]:
resto['accessibility'] = resto['accessibility'].replace({'partially':'yes',
                                                         'completely':'yes'})
resto['accessibility'].value_counts(1)

no_accessibility    0.616628
yes                 0.383372
Name: accessibility, dtype: float64

In [11]:
dict_cuis = {
     'Bar_Pub_Brewery':'Bar',
     'Seafood':'Other',
     'Japanese':'International',
     'Pizzeria':'Fast_Food',
     'Burgers':'Fast_Food',
     'American':'International',
     'Italian':'International',
     'Chinese':'International',
     'Contemporary':'Other',
     'Family':'Other',
     'Breakfast-Brunch':'Other',
     'Game':'Other',
     'Vietnamese':'International',
     'Armenian':'International',
     'Regional':'Mexican',
     'Bakery':'Other',
     'Cafe-Coffee_Shop':'Cafeteria',
     'Mediterranean':'International'
    }

resto['Rcuisine'] = resto['Rcuisine'].replace(dict_cuis).replace(np.nan,'?')
resto['Rcuisine'].value_counts(1, dropna = False)

?                0.242494
Mexican          0.196305
International    0.157044
Bar              0.131640
Fast_Food        0.120092
Other            0.083141
Cafeteria        0.069284
Name: Rcuisine, dtype: float64

In [12]:
resto.sample(4)

,latitude,longitude,city,alcohol,smoking_area,accessibility,price,franchise,area,Rcuisine,hours,days
placeID,,,,,,,,,,,,
134996,18.923429,-99.216413,cuernavaca,Wine-Beer,permitted,no_accessibility,medium,t,closed,?,07:00-23:00;,Sun;
132572,22.141647,-100.992712,slp,No_Alcohol_Served,not permitted,yes,low,f,closed,Cafeteria,00:00-23:30;,Mon;Tue;Wed;Thu;Fri;
135081,22.164842,-100.960493,slp,No_Alcohol_Served,not permitted,no_accessibility,medium,f,closed,?,14:00-23:30;,Sat;
135064,22.154687,-100.996617,slp,No_Alcohol_Served,not permitted,no_accessibility,high,f,closed,?,07:00-22:30;,Sat;


In [13]:
resto.reset_index(inplace = True)
resto = resto.groupby(list(resto.columns[:-1])
                     )['days'].sum().reset_index().set_index('placeID')

In [14]:
dim_antes = len(resto)

In [15]:
resto['days'] = [' '.join(np.unique(x)) for x in sorted(resto['days'].str.split(';'))]
resto['days'] = resto['days'].str.strip().str.lower()
resto['days'].value_counts(1)

fri mon sat sun thu tue wed    0.502703
sun                            0.162162
fri mon thu tue wed            0.113514
fri mon sat thu tue wed        0.108108
sat sun                        0.059459
sat                            0.054054
Name: days, dtype: float64

In [16]:
resto.dropna(inplace = True)
resto['open'] = pd.to_datetime(resto['hours'].str[:5])
resto['close'] = pd.to_datetime(resto['hours'].str[-6:].str[:-1])
resto['horario'] = (resto['close'] - resto['open']).dt.seconds // 3600
resto['open'] = resto['open'].dt.hour
resto['close'] = resto['close'].dt.hour
resto['horario'] = resto['horario']

resto['horario'].value_counts(1)

23    0.151351
0     0.102703
10    0.097297
11    0.075676
9     0.075676
12    0.064865
15    0.054054
13    0.054054
8     0.048649
6     0.048649
14    0.043243
7     0.043243
5     0.043243
16    0.037838
3     0.027027
4     0.021622
19    0.005405
22    0.005405
Name: horario, dtype: float64

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range = (1, 1), 
                     min_df = 10, 
                     max_features = 50)
texto = 'days'
cv.fit(resto[texto])
words = cv.get_feature_names()
resto = resto.join(pd.DataFrame(data = cv.transform(resto[texto]).todense(),
                                index = resto.index, 
                                columns = words)).drop(columns = ['days','hours'])
resto.sample(4)

,latitude,longitude,city,alcohol,smoking_area,accessibility,price,franchise,area,Rcuisine,open,close,horario,fri,mon,sat,sun,thu,tue,wed
placeID,,,,,,,,,,,,,,,,,,,,
132921,22.150305,-100.989134,?,Wine-Beer,permitted,yes,low,f,open,Bar,1,23,22,1,1,1,1,1,1,1
135057,22.145992,-100.955118,slp,Wine-Beer,permitted,no_accessibility,medium,f,closed,Bar,13,23,10,0,0,1,1,0,0,0
135065,22.149580,-100.999557,slp,Wine-Beer,permitted,no_accessibility,high,f,closed,?,14,18,4,0,0,0,1,0,0,0
135035,22.145813,-101.018032,slp,No_Alcohol_Served,permitted,no_accessibility,high,f,closed,International,13,23,10,1,1,1,1,1,1,1


grupo = ['latitude','longitude','fri', 'mon', 'sat', 'sun', 'thu', 'tue', 'wed']
resto = resto.reset_index().merge(resto.groupby(grupo)['horario'].max().reset_index(),
                                  on = grupo + ['horario'],
                                  how = 'inner').drop_duplicates().set_index('placeID')

for col in ['open','close','horario']:
    for weekday in ['mon','tue','wed','thu','fri','sat','sun']:
        resto[f'{col}_{weekday}'] = resto[col]*resto[weekday]
resto.sample(4)

In [18]:
num = [x for x in resto.describe().columns]
cat = [x for x in resto.columns if x not in num]
cat

['city',
 'alcohol',
 'smoking_area',
 'accessibility',
 'price',
 'franchise',
 'area',
 'Rcuisine']

## Ratings

In [19]:
ratings.sample(4)

,userID,placeID,rating,food_rating,service_rating
902,U1114,132834,0,0,0
368,U1007,135057,1,1,2
548,U1045,135032,0,0,0
1147,U1130,132561,0,0,0


In [20]:
matrix = ratings.pivot_table(index = 'placeID', 
                             columns = ['rating','food_rating','service_rating'], 
                             values = 'userID', 
                             aggfunc = 'count',
                             fill_value = 0)
matrix.columns = ['rating' + str(x) +
                  '_food' + str(y) + 
                  '_service' + str(z) for x,y,z in matrix.columns]

matrix.drop(columns = [col for col in matrix.columns if matrix[col].std() == 0],
            inplace = True)
matrix.sample(4)

,rating0_food0_service0,rating0_food0_service1,rating0_food0_service2,rating0_food1_service0,rating0_food1_service1,rating0_food2_service0,rating0_food2_service1,rating0_food2_service2,rating1_food0_service0,rating1_food0_service1,rating1_food0_service2,rating1_food1_service0,rating1_food1_service1,rating1_food1_service2,rating1_food2_service0,rating1_food2_service1,rating1_food2_service2,rating2_food0_service0,rating2_food0_service1,rating2_food0_service2,rating2_food1_service0,rating2_food1_service1,rating2_food1_service2,rating2_food2_service0,rating2_food2_service1,rating2_food2_service2
placeID,,,,,,,,,,,,,,,,,,,,,,,,,,
135109,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
135058,3,0,0,0,0,0,0,1,1,0,1,0,3,1,0,1,1,0,0,1,1,1,0,1,1,1
132594,1,0,0,0,0,1,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0
132767,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3


In [21]:
df = resto.join(matrix.astype(int))
df.drop(columns = ['latitude','longitude'], inplace = True)
df.columns

Index(['city', 'alcohol', 'smoking_area', 'accessibility', 'price',
       'franchise', 'area', 'Rcuisine', 'open', 'close', 'horario', 'fri',
       'mon', 'sat', 'sun', 'thu', 'tue', 'wed', 'rating0_food0_service0',
       'rating0_food0_service1', 'rating0_food0_service2',
       'rating0_food1_service0', 'rating0_food1_service1',
       'rating0_food2_service0', 'rating0_food2_service1',
       'rating0_food2_service2', 'rating1_food0_service0',
       'rating1_food0_service1', 'rating1_food0_service2',
       'rating1_food1_service0', 'rating1_food1_service1',
       'rating1_food1_service2', 'rating1_food2_service0',
       'rating1_food2_service1', 'rating1_food2_service2',
       'rating2_food0_service0', 'rating2_food0_service1',
       'rating2_food0_service2', 'rating2_food1_service0',
       'rating2_food1_service1', 'rating2_food1_service2',
       'rating2_food2_service0', 'rating2_food2_service1',
       'rating2_food2_service2'],
      dtype='object')

## Distancias

In [22]:
num = [x for x in df.describe().columns]
cat = [x for x in df.columns if x not in num]
df = df[cat + num].copy()

df.sample(4)

,city,alcohol,smoking_area,accessibility,price,franchise,area,Rcuisine,open,close,horario,fri,mon,sat,sun,thu,tue,wed,rating0_food0_service0,rating0_food0_service1,rating0_food0_service2,rating0_food1_service0,rating0_food1_service1,rating0_food2_service0,rating0_food2_service1,rating0_food2_service2,rating1_food0_service0,rating1_food0_service1,rating1_food0_service2,rating1_food1_service0,rating1_food1_service1,rating1_food1_service2,rating1_food2_service0,rating1_food2_service1,rating1_food2_service2,rating2_food0_service0,rating2_food0_service1,rating2_food0_service2,rating2_food1_service0,rating2_food1_service1,rating2_food1_service2,rating2_food2_service0,rating2_food2_service1,rating2_food2_service2
placeID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
135074,slp,Wine-Beer,permitted,no_accessibility,high,f,closed,Other,7,23,16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2
135013,jiutepec,No_Alcohol_Served,not permitted,no_accessibility,low,f,closed,?,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,2
135055,slp,No_Alcohol_Served,permitted,no_accessibility,high,f,closed,Mexican,14,22,8,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
135028,slp,Wine-Beer,not permitted,no_accessibility,medium,f,closed,Mexican,13,23,10,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,1,0,0,0,2,1,0,0,0,6


In [23]:
## Dummies para categóricas
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')

## Escala para numéricas
from sklearn.preprocessing import MinMaxScaler
mm_x = MinMaxScaler()

## Se aplicará transformación para cada tipo de columnas
from sklearn.compose import ColumnTransformer
prep = ColumnTransformer(transformers=[('OHE', ohe, cat),
                                       ('Scale', mm_x, num)])

In [24]:
df = pd.DataFrame(prep.fit_transform(df), index = df.index)
df = df.reset_index().groupby('placeID').mean()
df.sample(4)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62
placeID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
135033,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428571,0.913043,0.521739,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
135030,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.4,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.333333,0.250000
132560,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.190476,0.260870,0.086957,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.166667,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
135066,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,1.000000,0.695652,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,1.0,0.0,0.000000,0.166667


In [25]:
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('manhattan')
recom = pd.DataFrame(dist.pairwise(df), index = df.index, columns = df.index)

recom.sample(4)

placeID,132560,132561,132564,132572,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,132858,132861,132862,132866,132869,132870,132872,132875,132877,132884,132885,132921,132922,132925,132937,132951,132954,132955,132958,134975,134983,134986,134987,134992,134996,134999,135000,135001,135011,135013,135016,135018,135019,135021,135025,135026,135027,135028,135030,135032,135033,135034,135035,135038,135039,135040,135041,135042,135043,135044,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
placeID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
135032,20.740683,20.687681,19.771014,18.187681,20.119220,22.532195,22.842443,21.370221,17.787578,19.837681,20.237681,20.054348,19.837681,20.354348,18.637681,21.921014,19.304348,18.526398,18.957557,16.889441,13.736128,23.179400,17.674845,20.141097,13.605590,14.971014,13.804348,14.997239,16.419738,13.675466,17.805383,18.521014,13.648654,12.843168,13.341097,18.887164,16.069151,12.853727,14.326915,18.052070,15.239959,14.315528,11.102484,10.906625,16.066770,14.167909,16.021014,17.687681,17.971014,14.224431,14.446584,13.093168,13.322774,15.822257,15.487681,15.475155,15.837681,18.806625,15.721532,14.803830,14.421014,13.214838,12.966046,10.250000,14.004969,13.811594,14.916667,18.062319,17.162215,16.145652,17.578986,20.612319,19.721014,10.717357,14.102070,12.269358,10.537164,11.495652,0.000000,10.106108,11.963975,12.313975,9.039959,10.550828,11.507764,8.962319,12.138302,17.612319,12.466874,14.770497,17.375155,14.667909,18.734679,17.583023,20.969979,19.729089,19.024431,13.368012,17.081159,17.261594,13.346584,21.613251,14.705487,15.239027,16.171014,16.625052,19.070290,16.321739,13.850000,18.898861,17.266667,17.377019,17.925569,17.010352,14.116667,15.721014,18.722153,20.316874,17.387681,19.571014,18.908075,18.412319,19.857764,22.314700,25.004348,17.889027,21.771014,17.481677
132706,11.738095,15.180745,15.097412,17.280745,8.073844,12.660870,13.216046,9.851829,7.970704,9.030745,11.630745,12.814079,13.030745,11.114079,10.230745,9.947412,9.564079,0.000000,11.331159,10.703623,14.591511,14.453002,6.899172,12.843271,9.407764,9.497412,10.664079,16.811422,10.126674,17.215735,20.245652,17.347412,15.717081,9.310973,16.251967,14.927433,15.523913,14.834576,18.461387,15.519876,11.448344,18.110870,11.823913,10.381677,10.621532,15.331988,17.847412,15.180745,14.897412,14.368634,23.109213,14.394306,11.870290,19.762526,16.380745,9.943996,14.364079,12.881677,17.871532,15.431056,14.780745,12.753658,16.130124,12.676398,9.865528,10.042340,12.676398,9.226398,12.826294,11.809731,16.976398,15.943064,12.747412,9.657626,15.799482,12.657039,12.697723,14.126398,18.526398,12.486957,10.357867,13.964389,17.404037,15.224431,14.085300,15.326398,13.065528,11.343064,10.681056,17.797723,15.617909,20.525155,19.048861,14.556625,18.943582,21.626501,23.068634,17.415528,16.532195,17.155487,22.314389,19.047723,21.773292,19.312629,19.097412,15.046273,19.522774,23.138820,23.743064,22.027536,17.326398,21.814389,16.764389,22.907764,22.476398,18.747412,22.256625,21.431056,20.188820,19.272153,17.215010,23.076398,16.335300,20.212112,17.864079,23.169876,23.097412,17.711387
135046,20.260766,18.529503,18.612836,18.129503,16.710732,20.104865,23.667288,16.954210,19.326708,17.546170,19.146170,17.829503,17.546170,18.129503,17.079503,18.562836,16.746170,15.617909,16.282402,11.314286,18.527226,21.070911,14.843168,20.965942,11.363768,12.346170,15.346170,19.842374,16.244582,22.833644,19.896894,19.529503,14.852795,11.841925,16.767391,17.712008,15.467909,15.532195,19.405383,15.0508

## Recomendación

In [26]:
rest.sample(4)

,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,zip,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services,Rcuisine,hours,days
placeID,,,,,,,,,,,,,,,,,,,,,,,
135040,22.135617,-100.969709,0101000020957F00001B552189B84A58C15A2AAEFD2CA2...,Restaurant los Compadres,Camino a Simon Diaz 155 Centro,San Luis Potosi,SLP,Mexico,?,74000,Wine-Beer,none,informal,no_accessibility,high,?,familiar,f,closed,none,NaN,08:00-23:30;,Mon;Tue;Wed;Thu;Fri;
132851,22.136872,-100.934574,0101000020957F0000C74DE39C7F4B58C1061F29171498...,KFC,?,San Luis Potosi,San Luis Potosi,Mexico,?,?,No_Alcohol_Served,not permitted,informal,completely,low,?,familiar,f,closed,none,American,09:00-22:00;,Sat;
132870,22.143078,-100.935479,0101000020957F0000550FC1FC854A58C16D7D4E35F897...,Tortas y hamburguesas el gordo,Ricardo B. Anaya,San Luis Potosi,San Luis Potosi,Mexico,?,?,No_Alcohol_Served,not permitted,informal,no_accessibility,low,?,familiar,f,closed,none,Burgers,10:30-23:00;,Mon;Tue;Wed;Thu;Fri;
135000,18.870565,-99.226938,0101000020957F00001CCDD2577A775AC1D25FB8BA1F56...,Restaurant los Pinos,Federal Mexico - Acapulco 10 3 de Mayo,Cuernavaca,Morelos,Mexico,?,62790,No_Alcohol_Served,none,informal,no_accessibility,medium,?,familiar,f,closed,none,NaN,07:30-18:00;,Mon;Tue;Wed;Thu;Fri;


In [27]:
import mpu
def recomenda(n = 5):
    while True:
        try:
            name = input('Qué restaurante te gusta?: ')
            rest.reset_index().set_index('name').loc[name,'placeID'][0]
        except:
            print(f'El restaurante "{name}" no está en el catálogo, intenta de nuevo!\n')
            continue
        else:
            print(f'Estos {n} restaurantes podrían gustarte también: ')
            busca = rest.reset_index().set_index('name').loc[name,['placeID',
                                                                   'latitude',
                                                                   'longitude']]
            coor = (busca['latitude'][0],busca['longitude'][0])
            busca = busca['placeID'][0]
            vecinos = recom[busca].reset_index().sort_values(by = busca).iloc[1:n+1,0]
            for i,x in enumerate(vecinos):
                recomendacion = rest.loc[x,['name','latitude','longitude']].reset_index()
                cerca = (recomendacion['latitude'][0],recomendacion['longitude'][0])
                resultado = recomendacion['name'][0]
                distancia = mpu.haversine_distance(coor,cerca)
                print(f'\t #{i+1}. {resultado.capitalize()} (a {round(distancia,2)}km de distancia)')
            break

In [28]:
recomenda(7)

Qué restaurante te gusta?: Koye Sushi
Estos 7 restaurantes podrían gustarte también: 
	 #1. Sushi itto (a 0.93km de distancia)
	 #2. Restaurante y pescaderia tampico (a 8.7km de distancia)
	 #3. La estrella de dimas (a 8.59km de distancia)
	 #4. Restaurante pueblo bonito (a 3.74km de distancia)
	 #5. Restaurante el chivero s.a. de c.v. (a 0.26km de distancia)
	 #6. Los toneles (a 1.26km de distancia)
	 #7. Mariscos el pescador (a 2.0km de distancia)
